### Import

In [47]:
import os 
import openai
import pymysql
import pandas as pd
import datetime as dt
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv

### open api key 설정

In [48]:
load_dotenv()

True

In [49]:
key = os.environ.get("my-key")
openai.api_key=key

### DB 사용 예시

In [ ]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='modoostock', charset='utf8mb4', autocommit=True)
cur = con.cursor()

sql = "INSERT INTO table_name (value1, value2) VALUES (123, '이름');"
cur.execute(sql)

con.commit()
con.close()


### 마스킹 함수

#### 1. 기업명 가리기

In [50]:
def masking_companyName(content):
  check_list = ["SK텔레콤", "SK 텔레콤", "SKT", "SK T", "SK", "에스케이텔레콤", "에스케이"]
  for check in check_list:
    content = content.replace(check, "[]")

  return content

#### 2. OPEN AI API 사용

In [51]:
def masking_openai(content):
  MODEL = "gpt-3.5-turbo"
  prompt = """아래 글에서 각 기업명들을 XXX로 마스킹 처리해줘.
    {}
    """.format(content)

    # max_tokens 답변을 받을 수 있는 길이 (낮은 수로 설정해둔다면 말하다 끊김)
    
  response = openai.ChatCompletion.create(
      model=MODEL,
      messages=[
          {"role": "user", "content": prompt},
      ],
      temperature=0,
  )

  return response['choices'][0]['message']['content']

### Data 들고오기!

In [38]:
con = pymysql.connect(host='localhost', user='ssafy', password='ssafy', 
                      db='modoostock', charset='utf8mb4', autocommit=True)

In [39]:
result = pd.read_sql(sql="SELECT * FROM news WHERE company_id = 9;", con=con, index_col="id")
con.close()

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12924\2261292878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(sql="SELECT * FROM news WHERE company_id = 9;", con=con, index_col="id")


### 마스킹하기

#### 1. 기업명 가리기

In [40]:
result["content"] = result["content"].apply(masking_companyName)

#### 2. OPEN AI API 사용

In [44]:
result["content"][88057]

'[르포] 상생 현장을 가다/ [] 특수목적 단말 테스트센터'

In [53]:
result_one = masking_openai(result["content"][88057])

#### 3. 현재 기업명 복구

In [54]:
result_one.replace("[XXX]", "I IT")

'[르포] 상생 현장을 가다/ I IT 특수목적 단말 테스트센터'